In [437]:
import numpy as np

### Generate Demand Matrix

In [449]:
def generate_demand_matrix():
    """
    Output to modeldata.py
    ----------------------
    Data.inbound_demand_matrix: numpy.ndarray
        All zeros matrix for the inbound section of the demand matrix.
        Rows: Σ|C| (total number of customer across all products)
        Columns: Σ|F| (total number of factories across
        all products)
        Major order: 1.product, 2.factory, 3.customer

    Data.outbound_demand_matrix: numpy.ndarray
        Block diagonal matrix for the outbound section of the demand matrix.
        Rows: Σ|C| (total number of customer across all products)
        Columns: Σ|FxC| (total number of factories x customer across
        all products)

    Data.demand_matrix: numpy.ndarray
        Demand matrix to realize customers' demand

    """

    # Verify inputs type
    assert isinstance(Data.customer_sizes,
                      dict), 'Customer sizes must be a dictionary'
    assert isinstance(sum(Data.customer_sizes.values()),
                      int), 'Customer sizes must be integers'
    assert isinstance(Data.factory_sizes,
                      dict), 'Factory sizes must be a dictionary'
    assert isinstance(sum(Data.factory_sizes.values()),
                      int), 'Factory sizes must be integers'
    assert isinstance(Data.product_list,
                      list), 'Product list must be a list (Duhh!?)'

    # Verify inputs values
    assert np.all(np.array(list(Data.customer_sizes.values())) > 0
                  ), 'Customer sizes must be positive'

    assert np.all(np.array(list(Data.factory_sizes.values())) > 0
                  ), 'Factory sizes must be positive'

    # Verify input length
    assert (len(Data.customer_sizes) == len(Data.product_list)), \
        'Number of products in customer sizes is incorrect'

    assert (len(Data.factory_sizes) == len(Data.product_list)), \
        'Number of products in factory sizes is incorrect'

    # Reshape dictionary inputs into matrices
    # Construct zero inbound cost matrix (Σ|C|, Σ|F|)
    Data.inbound_demand_matrix = np.zeros(
        (Data.dimC, Data.dimF))  # Demand Inbound Block

    # Construct block diagonal outbound cost matrix (Σ|C|, Σ|FxC|)
    Data.outbound_demand_matrix = block_diag(*[
        np.tile(np.eye(Data.customer_sizes[product]),
                reps=Data.factory_sizes[product])
        for product in Data.product_list
    ])  # Demand Outbound Block

    # Verify output dimension
    # Outbound demand matrix dimension = (Σ|C|, Σ|FxC|)
    assert Data.outbound_demand_matrix.shape == (Data.dimC, Data.dimFC)

    # Horizontally stack inbound and outbound block into the full demand matrix
    Data.demand_matrix = np.hstack(
        [Data.inbound_demand_matrix, Data.outbound_demand_matrix])

    # Verify non-negative
    assert np.all(
        Data.demand_matrix >= 0), 'Demand matrix must be non-negative'

### Unit Test

To unit the test the demand matrix, we will random generate a list of factory customer combination for each product. Then will iterate over the matrix to find if a particular combination is presence. We will also check that it's all that there is presence. We do this by first filling in the the matrix with zeros, then delete the all zeros rows and columns. Then we compare it with the first demand matrix

Realisitically, we will have repeat for both customer and factory in the product

In [502]:
import unittest
import numpy as np
from scipy.linalg import block_diag
import sys  # Get the path to the "model" directory

sys.path.append("C:\\Users\\monty.minh\\Documents\\Model4.0")

from model.modeldata import Data
from model.optimization import generate_demand_matrix


class CombinationTest:
    """Class for generating random test inputs and verify that the demand
    matrix is constructed correctly"""

    no_products = None
    factory_names = None
    customer_names = None
    no_factories = None
    no_customers = None

    @classmethod
    def generate_random_inputs(cls):
        """Generate random Data inputs"""

        cls.no_customers = np.random.randint(1, 10)
        cls.no_factories = np.random.randint(1, 5)
        cls.no_products = np.random.randint(1, 5)

        if cls.no_customers == 1:
            cls.customer_names = dict(zip(range(cls.no_products), [np.array([0])] * cls.no_products))
        else: # Allow for one customer
            cls.customer_names = {
                prod: np.sort(
                    np.random.choice(np.arange(cls.no_customers),
                                     size=np.random.randint(1, cls.no_customers),
                                     replace=False))
                for prod in range(cls.no_products)
            }

        if cls.no_factories == 1:
            cls.factory_names = dict(zip(range(cls.no_products), [np.array([0])] * cls.no_products))
        else: # Allow for one factory
            cls.factory_names = {
                prod: np.sort(
                    np.random.choice(np.arange(cls.no_factories),
                                     size=np.random.randint(1, cls.no_factories),
                                     replace=False))
                for prod in range(cls.no_products)
            }

        # Inputs from Data
        Data.customer_sizes = {
            prod: len(cls.customer_names[prod])
            for prod in range(cls.no_products)
        }

        Data.factory_sizes = {
            prod: len(cls.factory_names[prod])
            for prod in range(cls.no_products)
        }

        Data.product_list = list(range((cls.no_products)))

        Data.dimF = sum(Data.factory_sizes.values())
        Data.dimC = sum(Data.customer_sizes.values())

        Data.dimFC = sum([
            Data.factory_sizes[prod] * Data.customer_sizes[prod]
            for prod in range(cls.no_products)
        ])

    @classmethod
    def alternative_demand_matrix(cls):
        """Construct the demand matrix in an alternative method"""

        block_list = []

        for prod in Data.product_list:
            block = np.zeros(
                (cls.no_customers, cls.no_customers * cls.no_factories))

            # Row Index
            row_index = np.repeat(cls.customer_names[prod],
                                  repeats=Data.factory_sizes[prod])

            # Column Index
            column_index = np.hstack(cls.no_customers *
                                     cls.factory_names[prod] +
                                     cls.customer_names[prod][:, np.newaxis])

            # Fill ones in the appropriate place
            block[row_index, column_index] = 1

            # Remove columns and rows with all zeros
            block = block[:, ~np.all(block ==
                                     0, axis=0)][~np.all(block == 0, axis=1)]

            block_list.append(block)

        # Append with the inbound block then return
        return np.hstack(
            [np.zeros((Data.dimC, Data.dimF)),
             block_diag(*block_list)])


class TestDemandMatrix(unittest.TestCase):
    def test_demand_matrix(self):
        for _ in range(100):
            CombinationTest.generate_random_inputs()  # Random Inputs
            generate_demand_matrix()  # Generate Demand Matrix

            self.assertTrue(
                np.array_equal(CombinationTest.alternative_demand_matrix(),
                               Data.demand_matrix)
            )  # Compare the demand matrix, with an alternative method


if __name__ == '__main__':
    unittest.main()